#### Preparations 

In [1]:
# All needed libraries:
library(tidyverse)
library(tidymodels)
library(broom)
library(repr)
library(digest)
library(infer)
library(gridExtra)
library(cowplot)
library(leaps)
library(glmnet)
library(car)
library(faraway)
library(mltools)

# General Graphs' setting:
options(repr.plot.width = 13, repr.plot.height = 9)   

# Import online dataset
url <- "https://raw.githubusercontent.com/Jitao-Z/Medical-Cost-Personal-Datasets/main/insurance.csv"
raw_data <- read_csv(url)

# First 6 rows of the dataset
head(raw_data)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

age,sex,bmi,children,smoker,region,charges
<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
19,female,27.900,0,yes,southwest,16884.924
18,male,33.770,1,no,southeast,1725.552
28,male,33.000,3,no,southeast,4449.462
33,male,22.705,0,no,northwest,21984.471
32,male,28.880,0,no,northwest,3866.855
31,female,25.740,0,no,southeast,3756.622


#### Data cleaning

In [2]:
medical_cost <- raw_data |>
    filter(!is.na(charges)) |>                        # Check and remove missing values from the raw dataset
    mutate(region = ifelse(region == "southwest" | region == "northwest", "west", "east"))
medical_cost$sex <- factor(medical_cost$sex)          # Change the property of sex into factor
medical_cost$region <- factor(medical_cost$region)    # Change the property of region into factor

head(medical_cost)

age,sex,bmi,children,smoker,region,charges
<dbl>,<fct>,<dbl>,<dbl>,<chr>,<fct>,<dbl>
19,female,27.900,0,yes,west,16884.924
18,male,33.770,1,no,east,1725.552
28,male,33.000,3,no,east,4449.462
33,male,22.705,0,no,west,21984.471
32,male,28.880,0,no,west,3866.855
31,female,25.740,0,no,east,3756.622


#### Ridge, Full regression, Lasso

In [3]:
# Set seed to obtain reproducible results
set.seed(1234)

# Create training and testing sets
medical_split <- initial_split(medical_cost, prop = 0.7, strata = charges)
medical_training <- training(medical_split)
medical_testing <- testing(medical_split)

# Change training and testing sets into matrix form required by "glmnet( )"
medical_matrix_X_train <- model.matrix(charges ~ ., medical_training)[, -1]

medical_matrix_Y_train <- as.matrix(medical_training$charges, ncol = 1)

medical_matrix_X_test <- model.matrix(charges ~ ., medical_testing)[, -1]

medical_matrix_Y_test <- as.matrix(medical_testing$charges, ncol = 1)

- Ridge

In [4]:
# Provide different levels of lambda on the model to obtain their respective MSE
medical_cv_lambda_ridge <- cv.glmnet(
    x = medical_matrix_X_train, y = medical_matrix_Y_train,
    alpha = 0)               # alpha = 0: Ridge penalty
                             # Here, we do not explicitly set up a range of values for lambda; therefore, the function will use its defalut argument of lambda for us
medical_cv_lambda_ridge


Call:  cv.glmnet(x = medical_matrix_X_train, y = medical_matrix_Y_train,      alpha = 0) 

Measure: Mean-Squared Error 

    Lambda Index  Measure      SE Nonzero
min  970.5   100 38377820 2738198       6
1se 2460.7    90 40934538 3176953       6

In [5]:
# Obtain the lambda that provides the smallest MSE
medical_lambda_min_MSE_ridge <- medical_cv_lambda_ridge$lambda.min
medical_lambda_min_MSE_ridge

[1] 970.5486

In [6]:
# Obtain the estimated coefficients of our regularized model by selecting the lambda which provides the smallest MSE
medical_ridge_min_coef <- coef(medical_cv_lambda_ridge, s = "lambda.min")    
medical_ridge_min_coef

7 x 1 sparse Matrix of class "dgCMatrix"
                     s1
(Intercept) -8932.33175
age           225.86735
sexmale      -108.60677
bmi           270.71052
children      515.48543
smokeryes   22480.77443
regionwest     18.81022

In [7]:
# Apply our regularized model on our testing set to predict the corresponding response variable, charges
medical_test_pred_ridge_min <- predict(medical_cv_lambda_ridge, newx = medical_matrix_X_test, s = "lambda.min")
head(medical_test_pred_ridge_min)

,lambda.min
1,25411.556
2,4682.054
3,7763.251
4,6023.747
5,3703.775
6,34669.198


In [8]:
# Calculate the RMSE and present our final result in a table 
medical_RMSE <- tibble(
    Model = "Ridge Regression with minimum MSE",
    RMSE = rmse(preds = medical_test_pred_ridge_min, actuals = medical_matrix_Y_test))
medical_RMSE

Model,RMSE
<chr>,<dbl>
Ridge Regression with minimum MSE,6040.751


- Lasso

In [9]:
# Provide different levels of lambda on the model to obtain their respective MSE
medical_cv_lambda_LASSO <- cv.glmnet(
    x = medical_matrix_X_train, y = medical_matrix_Y_train,
    alpha = 1)               # alpha = 1: Lasso penalty
                             # Here, we do not explicitly set up a range of values for lambda; therefore, the function will use its defalut argument of lambda for us
medical_cv_lambda_LASSO


Call:  cv.glmnet(x = medical_matrix_X_train, y = medical_matrix_Y_train,      alpha = 1) 

Measure: Mean-Squared Error 

    Lambda Index  Measure      SE Nonzero
min  111.6    49 37869430 2080470       5
1se  787.2    28 39847674 2376031       3

In [10]:
# Obtain the lambda that provides the smallest MSE
medical_lambda_min_MSE_LASSO <- medical_cv_lambda_LASSO$lambda.min
medical_lambda_min_MSE_LASSO

[1] 111.5895

In [11]:
# Obtain the estimated coefficients of our regularized model by selecting the lambda which provides the smallest MSE
medical_LASSO_min_coef <- coef(medical_cv_lambda_LASSO, s = "lambda.min")    
medical_LASSO_min_coef

7 x 1 sparse Matrix of class "dgCMatrix"
                      s1
(Intercept) -9637.237289
age           236.560139
sexmale        -9.198455
bmi           270.862338
children      460.328206
smokeryes   23984.833691
regionwest      .       

In [12]:
# Apply our regularized model on our testing set to predict the corresponding response variable, charges
medical_test_pred_LASSO_min <- predict(medical_cv_lambda_LASSO, newx = medical_matrix_X_test, s = "lambda.min")
head(medical_test_pred_LASSO_min)

,lambda.min
1,26399.298
2,4218.996
3,7296.690
4,5745.993
5,3369.578
6,36135.296


In [13]:
# Calculate the RMSE and present our final result in a table 
medical_RMSE <- medical_RMSE |> add_row(
    Model = "Lasso Regression with minimum MSE",
    RMSE = rmse(preds = medical_test_pred_LASSO_min, actuals = medical_matrix_Y_test))
medical_RMSE

Model,RMSE
<chr>,<dbl>
Ridge Regression with minimum MSE,6040.751
Lasso Regression with minimum MSE,6035.074


- Full regression

In [14]:
# We can also fit a full linear regression model on our training set
medical_full_OLS <- lm(charges ~ ., data = medical_training)

In [15]:
# At this point, we can compare the estimated coefficients of all of our three obtained models
medical_reg_coef <- round(cbind(
    Full_OLS = coef(medical_full_OLS),
    Ridge_min = as.vector(medical_ridge_min_coef), 
    Lasso_min = as.vector(medical_LASSO_min_coef)), 1) |> as.data.frame()
medical_reg_coef

,Full_OLS,Ridge_min,Lasso_min
,<dbl>,<dbl>,<dbl>
(Intercept),-10557.9,-8932.3,-9637.2
age,243.4,225.9,236.6
sexmale,-268.8,-108.6,-9.2
bmi,289.0,270.7,270.9
children,548.5,515.5,460.3
smokeryes,24307.2,22480.8,23984.8
regionwest,129.8,18.8,0.0


In [16]:
# Apply our full linear regression model on our testing set to predict the corresponding response variable, charges
medical_test_full_OLS <- predict(medical_full_OLS, medical_testing)

In [17]:
# Calculate the RMSE of the full linear regression model and add it to our previous result table
medical_RMSE <- medical_RMSE |>
    add_row(Model = "OLS Full Regression",
            RMSE = rmse(preds = medical_test_full_OLS, actuals = medical_testing$charges))
medical_RMSE

Model,RMSE
<chr>,<dbl>
Ridge Regression with minimum MSE,6040.751
Lasso Regression with minimum MSE,6035.074
OLS Full Regression,6031.042
